In [6]:
# RANDOM WALK NEIGHBORHOOD OF TOP GENES
#
#
# DOES NOT WORK RIGHT NOW
#
#
#
# input file: top genes 
#
import os
import sys
import numpy as np
import scipy.stats as st
import networkx as nx
import itertools as it
import random as rd
import pymysql
import pickle 
import os.path
from collections import defaultdict
import time
from sklearn.preprocessing import normalize
import basic_network_things as bnt

ID_symbol={}
symbol_ID={}
f=open('../data/symbols2entrezIDs.txt','r')
for line in f:
    if line[0]=='#':
        continue
    data      = line.strip().split('\t')
    symbol    = data[0]
    ID        = data[1]
    ID_symbol[ID]=symbol
    symbol_ID[symbol]=ID
f.close()

#
# define input files
#
#
r=0.9 # restart probability
alpha = 1
infile = '../../IBD_paper/genes/top_10_genes_2019_nov_row_IDs.txt'
network = '../network_repository/allsmallnetworks/DataS1_interactome_ppi.txt'
def rnd_walk_matrix(A, r, alpha, num_nodes):
    
    n = num_nodes
    M = normalize(A, norm='l1', axis=0)                                 # column wise normalized MArkov matrix
    factor = float((1-alpha)/num_nodes)
#     print(factor)
    E = np.multiply(factor,np.ones([num_nodes,num_nodes]))              # prepare 2nd scaling term
    M2 = np.multiply(alpha,M) + E                                       # mixture of Markov chains
    del M
    del E
    
    U = np.identity(n,dtype=int) 
    H = (1-r)*M2
    H1 = np.subtract(U,H)
    del U
    del M2
    del H    

    W = r*np.linalg.inv(H1)                                             # calculate random-walk matrix
    del H1
    return W


fnet = open(network, 'r')
G0 = bnt.read_network(network)
G = max(nx.connected_component_subgraphs(G0), key=len)
if G.number_of_nodes() < G0.number_of_nodes():
    print('Network is NOT connected! Giant component taken.')
else:
    print('Network is connected.')
            
fnet.close()
print('Got alpha=%.2f and as network: %s.' %(alpha,network))
# print('original network: #nodes=%s , # edges=%s ' %(G0.number_of_nodes(),G0.number_of_nodes()))
print('lcc network: #nodes=%s , # edges=%s ' %(G.number_of_nodes(),G.number_of_edges()))

# ###########################################################################
#                                                                           #
#    CALCULATES MATRIX INVERSION FOR GIVEN PARAMETERS                       #
#                                                                           #
# ###########################################################################
num_nodes = G.number_of_nodes()                                             
A = nx.adjacency_matrix(G, sorted(G.nodes()))
t0 = time.time()
W = rnd_walk_matrix(A, r, alpha, num_nodes)    
print('Inversion done, with r== %s' %(r))    

#    GENERATE DICT FOR NODE-LABELS TO INTEGERS
d_idx_entz = {}                                                             #
cc = 0                                                                      #
for entz in sorted(G.nodes()):                                              #
    d_idx_entz[cc] = entz                                                   #
    cc += 1                                                                 #
    d_entz_idx = dict((y,x) for x,y in d_idx_entz.items())                  #
print('rnd walk computing time: %.2f' %float(time.time()-t0)) 
# LOAD EXTERNAL GENE SET
geneset=bnt.read_gene_file_to_one(infile)
print('number of genes in set: %s' %len(geneset))

to_analyze=geneset&(G.nodes())
print('number of set genes on network: %s' %len(to_analyze))
for l_ent in to_analyze:
    nodeset = l_ent
    print(nodeset)
    p0 = np.zeros(G.number_of_nodes())
    # generate start vector
    for n in range(len(p0)):
        if n==nodeset:
            p0[n] = 1.
    print('check for sum of initial p-vec')
    print(np.sum(p0))
    pinf =  np.array(W.dot(p0))
    
    # DICT WITH NODE ID AND PVIS
    d_n_p = {}
    i = 0
    for x in pinf[0]:
        #     print(i,x)
        d_n_p[d_idx_entz[i]] = x/1
        i += 1
    # write file
    try:
        sym1=ID_symbol[str(l_ent)]
    except:
        sym1=str(l_ent)
        
    outfile = '../../IBD_paper/rwr/from_TOP_genes/ppi_2015_r%s_%s.1f.txt' %(r,sym1)


> done loading network:
> network contains 13460 nodes and 141296 links
Network is NOT connected! Giant component taken.
Got alpha=1.00 and as network: ../network_repository/allsmallnetworks/DataS1_interactome_ppi.txt.
lcc network: #nodes=13329 , # edges=141150 
Inversion done, with r== 0.9
rnd walk computing time: 176.53

> done reading gene sets:
> 10 genes found
number of genes in set: 10
number of set genes on network: 10
11261
check for sum of initial p-vec
0.0
26191
check for sum of initial p-vec
0.0
4772
check for sum of initial p-vec
0.0
260425
check for sum of initial p-vec
0.0
10666
check for sum of initial p-vec
0.0
3683
check for sum of initial p-vec
0.0
5771
check for sum of initial p-vec
0.0
9267
check for sum of initial p-vec
0.0
6776
check for sum of initial p-vec
0.0
1437
check for sum of initial p-vec
0.0
